# 1、使用FUNCTION_CALL模式

In [1]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
# 获取Tavily搜索的实例
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType, create_tool_calling_agent, AgentExecutor
from langchain.tools import Tool
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
dotenv.load_dotenv()

# 读取配置文件的信息
os.environ['TAVILY_API_KEY'] = "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"

# 获取Tavily搜索工具的实例
search = TavilySearchResults(max_results=3)

# 获取一个搜索的工具
# 使用Tool
search_tool = Tool(
    func=search.run,
    name="Search",
    description="用于检索互联网上的信息",
)


# 获取大语言模型
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)
# 提供提示词模板（以ChatPromptTemplate为例）
prompt_template = ChatPromptTemplate.from_messages([
    ("system","你是一个乐于助人的ai助手，根据用户的提问，必要时调用Search工具，使用互联网检索数据"),
    ("human","{input}"),
    ("system","{agent_scratchpad}"),
    # ("placeholder","{agent_scratchpad}")
])

# 获取Agent的实例：create_tool_calling_agent()
agent = create_tool_calling_agent(
    llm=llm,
    prompt=prompt_template,
    tools=[search_tool]
)

# 获取AgentExecutor的实例
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True,
)


# 通过AgentExecutor的实例调用invoke(),得到响应
result = agent_executor.invoke({"input":"查询今天北京的天气情况"})

# 处理响应
print(result)




> Entering new AgentExecutor chain...

Invoking: `Search` with `北京天气 2023年10月5日`


[{'title': '北京历史天气预报2023年10月份', 'url': 'https://www.tianqihoubao.com/lishi/beijing/month/202310.html', 'content': '| 日期 | 天气状况(白天/夜间) | 最高/最低气温 | 风力风向(白天/夜间) |\n ---  --- |\n| 2023年10月01日 | 晴 / 晴 | 26℃ / 12℃ | 北风 1-3级 / 北风 1-3级 |\n| 2023年10月02日 | 多云 / 多云 | 25℃ / 14℃ | 北风 1-3级 / 北风 1-3级 |\n| 2023年10月03日 | 多云 / 多云 | 25℃ / 15℃ | 北风 1-3级 / 北风 1-3级 |\n| 2023年10月04日 | 晴 / 晴 | 24℃ / 9℃ | 北风 1-3级 / 北风 1-3级 |\n| 2023年10月05日 | 多云 / 多云 | 22℃ / 12℃ | 北风 1-3级 / 北风 1-3级 |\n| 2023年10月06日 | 多云 / 多云 | 22℃ / 12℃ | 北风 1-3级 / 北风 1-3级 |\n| 2023年10月07日 | 多云 / 小雨 | 22℃ / 11℃ | 北风 1-3级 / 北风 1-3级 | [...] 前一月 后一月 北京历史天气 北京10月份天气统计信息  \n 搜索"城市名+天气后报"访问本站（北京天气后报）导出Excel\n\n### 天气状况统计\n\n2023年10月份北京北京天气状况统计图\n白天天气\n夜间天气\n全天综合\n\n## 其他月份\n\n### 风力分布分析\n\n2023年10月份北京风力等级和风向分布情况图 \n白天风力风向\n夜间风力风向\n全天综合\n\n### 关于天气后报\n\n天气后报提供全国34个省市所属的3146个地区的历史天气预报查询，数据来源于城市当天的天气预报信息，可以查询到历史天气气温，历史风向，历史风力等历史天气状况。\n\n### 快速链接\n\n### 联系我们\n\n关于我们 | 免责

注意：agent_scratchpad必须声明，它用于存储和传递Agent的思考过程。比如，在调用链式工具时（如先搜索天气再推荐行程），`agent_scratchpad` 保留所有历史步骤，避免上下文丢失。format方法会将intermediate_steps转换为特定格式的字符串，并赋值给agent_scratchpad变量。如果不传递intermediate_steps参数，会导致KeyError: 'intermediate_steps'错误。

# 2、使用ReAct模式

举例1：使用PromptTemplate实现

In [36]:
from langchain.agents import create_react_agent
from langchain_core.prompts import ChatPromptTemplate
# 获取Tavily搜索的实例
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType, create_tool_calling_agent, AgentExecutor
from langchain.tools import Tool
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
dotenv.load_dotenv()

# 读取配置文件的信息
os.environ['TAVILY_API_KEY'] = "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"

# 获取Tavily搜索工具的实例
search = TavilySearchResults(max_results=3)

# 获取一个搜索的工具
# 使用Tool
search_tool = Tool(
    func=search.run,
    name="Search",
    description="用于检索互联网上的信息",
)


# 获取大语言模型
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)
# 提供提示词模板（以PromptTemplate为例）
template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""
prompt_template = PromptTemplate.from_template(
    template=template,
)

# 获取Agent的实例：create_react_agent()
agent = create_react_agent(
    llm=llm,
    prompt=prompt_template,
    tools=[search_tool]
)

# 获取AgentExecutor的实例
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True,
)


# 通过AgentExecutor的实例调用invoke(),得到响应
result = agent_executor.invoke({"input":"查询今天北京的天气情况"})

# 处理响应
print(result)




> Entering new AgentExecutor chain...
我需要查找今天北京的天气情况。  
Action: Search  
Action Input: "北京天气情况 2023年10月25日"  [{'title': '北京历史天气预报2023年10月份', 'url': 'https://www.tianqihoubao.com/lishi/beijing/month/202310.html', 'content': '| 2023年10月24日 | 晴 / 晴 | 23℃ / 9℃ | 无持续风向 1-3级 / 无持续风向 1-3级 |\n| 2023年10月25日 | 晴 / 晴 | 25℃ / 10℃ | 无持续风向 1-3级 / 无持续风向 1-3级 |\n| 2023年10月26日 | 晴 / 晴 | 21℃ / 6℃ | 无持续风向 1-3级 / 无持续风向 1-3级 |\n| 2023年10月27日 | 晴 / 晴 | 23℃ / 7℃ | 无持续风向 1-3级 / 无持续风向 1-3级 |\n| 2023年10月28日 | 晴 / 晴 | 23℃ / 9℃ | 无持续风向 1-3级 / 无持续风向 1-3级 |\n| 2023年10月29日 | 晴 / 晴 | 19℃ / 11℃ | 无持续风向 1-3级 / 无持续风向 1-3级 |\n| 2023年10月30日 | 雾 / 多云 | 22℃ / 11℃ | 无持续风向 1-3级 / 无持续风向 1-3级 |\n| 2023年10月31日 | 晴 / 雾 | 22℃ / 12℃ | 无持续风向 1-3级 / 无持续风向 1-3级 | [...] # 北京历史天气预报 2023年10月份\n\n### 温度趋势分析\n\n2023年10月份北京市每日最高气温和最低气温变化趋势图 [...] 前一月 后一月 北京历史天气 北京10月份天气统计信息  \n 搜索"城市名+天气后报"访问本站（北京天气后报）导出Excel\n\n### 天气状况统计\n\n2023年10月份北京北京天气状况统计图\n白天天气\n夜间天气\n全天综合\n\n## 其他月份\n\n### 风力分布分析\n\n2023年10月份北京风力等级和风向分布情况图 \n白天风力风向\n夜间风力风向\n全天综合\

上述的举例1也可以改写为：

In [37]:
from langchain import hub
from langchain.agents import create_react_agent
from langchain_core.prompts import ChatPromptTemplate
# 获取Tavily搜索的实例
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType, create_tool_calling_agent, AgentExecutor
from langchain.tools import Tool
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
dotenv.load_dotenv()

# 读取配置文件的信息
os.environ['TAVILY_API_KEY'] = "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"

# 获取Tavily搜索工具的实例
search = TavilySearchResults(max_results=3)

# 获取一个搜索的工具
# 使用Tool
search_tool = Tool(
    func=search.run,
    name="Search",
    description="用于检索互联网上的信息",
)


# 获取大语言模型
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)
# 使用LangChain Hub中的官方ReAct提示模板
prompt_template = hub.pull("hwchase17/react")


# 获取Agent的实例：create_react_agent()
agent = create_react_agent(
    llm=llm,
    prompt=prompt_template,
    tools=[search_tool]
)

# 获取AgentExecutor的实例
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True,
)


# 通过AgentExecutor的实例调用invoke(),得到响应
result = agent_executor.invoke({"input":"查询今天北京的天气情况"})

# 处理响应
print(result)


D:\developTools\miniconda3\envs\pyth310\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
我需要查找今天北京的天气情况。  
Action: Search  
Action Input: '北京天气预报 2023年10月5日'  [{'title': '北京历史天气预报2023年10月份', 'url': 'https://www.tianqihoubao.com/lishi/beijing/month/202310.html', 'content': '| 日期 | 天气状况(白天/夜间) | 最高/最低气温 | 风力风向(白天/夜间) |\n ---  --- |\n| 2023年10月01日 | 晴 / 晴 | 26℃ / 12℃ | 北风 1-3级 / 北风 1-3级 |\n| 2023年10月02日 | 多云 / 多云 | 25℃ / 14℃ | 北风 1-3级 / 北风 1-3级 |\n| 2023年10月03日 | 多云 / 多云 | 25℃ / 15℃ | 北风 1-3级 / 北风 1-3级 |\n| 2023年10月04日 | 晴 / 晴 | 24℃ / 9℃ | 北风 1-3级 / 北风 1-3级 |\n| 2023年10月05日 | 多云 / 多云 | 22℃ / 12℃ | 北风 1-3级 / 北风 1-3级 |\n| 2023年10月06日 | 多云 / 多云 | 22℃ / 12℃ | 北风 1-3级 / 北风 1-3级 |\n| 2023年10月07日 | 多云 / 小雨 | 22℃ / 11℃ | 北风 1-3级 / 北风 1-3级 | [...] 前一月 后一月 北京历史天气 北京10月份天气统计信息  \n 搜索"城市名+天气后报"访问本站（北京天气后报）导出Excel\n\n### 天气状况统计\n\n2023年10月份北京北京天气状况统计图\n白天天气\n夜间天气\n全天综合\n\n## 其他月份\n\n### 风力分布分析\n\n2023年10月份北京风力等级和风向分布情况图 \n白天风力风向\n夜间风力风向\n全天综合\n\n### 关于天气后报\n\n天气后报提供全国34个省市所属的3146个地区的历史天气预报查询，数据来源于城市当天的天气预报信息，可以查询到历史天气气温，历史风向，历史风力等历史天气状况。\n\n### 快速链接

举例2：使用ChatPromptTemplate实现

In [38]:
from langchain.agents import create_react_agent
from langchain_core.prompts import ChatPromptTemplate
# 获取Tavily搜索的实例
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType, create_tool_calling_agent, AgentExecutor
from langchain.tools import Tool
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
dotenv.load_dotenv()

# 读取配置文件的信息
os.environ['TAVILY_API_KEY'] = "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"

# 获取Tavily搜索工具的实例
search = TavilySearchResults(max_results=3)

# 获取一个搜索的工具
# 使用Tool
search_tool = Tool(
    func=search.run,
    name="Search",
    description="用于检索互联网上的信息",
)


# 获取大语言模型
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)
# 提供提示词模板（以ChatPromptTemplate为例）
system_template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!"""

prompt_template = ChatPromptTemplate.from_messages([
    ("system",system_template),
    ("human","{input}"),
    ("system","{agent_scratchpad}"),
])

# 获取Agent的实例：create_react_agent()
agent = create_react_agent(
    llm=llm,
    prompt=prompt_template,
    tools=[search_tool]
)

# 获取AgentExecutor的实例
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True,
)


# 通过AgentExecutor的实例调用invoke(),得到响应
result = agent_executor.invoke({"input":"查询今天北京的天气情况"})

# 处理响应
print(result)




> Entering new AgentExecutor chain...
Question: 查询今天北京的天气情况
Thought: 我需要查找今天北京的天气信息。
Action: Search
Action Input: "北京天气 2023年10月27日"[{'title': '天气预报（2023年10月27日17时发布） - 北京市房山区人民政府', 'url': 'https://www.bjfsh.gov.cn/bsfw2/tdrqfw/cjr/tqybl/202310/t20231027_40067813.shtml', 'content': '# å¤©æ°\x94é¢\x84æ\x8a¥ï¼\x882023å¹´10æ\x9c\x8827æ\x97¥17æ\x97¶å\x8f\x91å¸\x83ï¼\x89\n\nå¤©æ°\x94é¢\x84æ\x8a¥ï¼\x882023å¹´10æ\x9c\x8827æ\x97¥17æ\x97¶å\x8f\x91å¸\x83ï¼\x89\n\næ\x97¥æ\x9c\x9f:2023-10-27 16:49    \næ\x9d¥æº\x90:å\x8cºæ°\x94è±¡å±\x80\n\nã\x80\x80ã\x80\x80ä»\x8aå¤©å¤\x9cé\x97´ï¼\x9a\n\nã\x80\x80ã\x80\x80æ\x99´ï¼\x8cæ\x9c\x89è½»é\x9b¾ï¼\x9b\n\nã\x80\x80ã\x80\x80å\x8d\x97è½¬å\x8c\x97é£\x8e1ã\x80\x812çº§ï¼\x9b\n\nã\x80\x80ã\x80\x80æ\x9c\x80ä½\x8eæ°\x94æ¸©ï¼\x9a6â\x84\x83ã\x80\x82\n\nã\x80\x80ã\x80\x80æ\x98\x8eå¤©ç\x99½å¤©ï¼\x9a\n\nã\x80\x80ã\x80\x80æ\x99´ï¼\x9b\n\nã\x80\x80ã\x80\x80å\x8c\x97è½¬å\x8d\x97é£\x8e2ã\x80\x813çº§ï¼\x9b\n\nã\x80\x80ã\x80\x80æ\x9c\x80é«\x98æ°\x94æ¸©ï¼\x9a23â\x84\x83ã\x80\

小结：

1、传统方式，相较于通用方式来讲，不用提供提示词模板。

2、对于通用方式来讲，

FUNCTION_CALL模式：在创建Agent时，推荐大家使用ChatPromptTemplate

ReAct模式：在创建Agent时，大家可以使用ChatPromptTemplate、PromptTemplate。但相较来讲，推荐大家使用PromptTemplate